In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import random

In [3]:
#define functions with 4 parameters
def sigmoid4(x,a,b,c,d):           
    return a+b/(1+np.exp(c*(x+d)))
def arctan4(x,a,b,c,d):   
    return a +(2/np.pi)*b * np.arctan(c*(x+d))
def erf4(x,a,b,c,d):     
    return a+b*sp.special.erf(c*(x+d)) 
def tanhyp4(x,a,b,c,d):     #This is mathematically the same as a logistic (sigmoid) function!. Use the result for testing
    return a+b*np.tanh(c*(x+d))

funcs4 = [sigmoid4,arctan4,erf4,tanhyp4]
vec_funcs4=[*map(np.vectorize,funcs4)]

In [ ]:
#initial parameter guesses for fits. The fitting routine can get stuck in local minima.
#trial and error is required to find initial parameters that lead to good fits.
#rough initial parameters can be estimated by physical considerations: e.g. start at 0.5 AUC, increase with training size, etc.
#columns are: trait,left asymptotic value,inflection pt,right asymptotic guess, slope at inflection point 
inits= pd.DataFrame([
["asthma",.5,4,.65,2], #done
 ["atrial.fibrillation",.5,3,.65,2], 
 ['breast',.5,3,.65,2], 
 ['breast-lou',.5,3,.65,2], 
 ["diabetes.type2",.5,3,.65,2], 
 ["diabetes.type1",.5,3,.65,2], 
 ['CAD',.5,3,.65,2], 
 ["hypertension",0.5, 4.5,.65,3], 
 ["hyperlip-cont",0.5, 4.5,.65,3], 
 ['gout',0.5,3.5,.7,2],
 ["Direct.bilirubin",.05,3,.45,2],
 ['BMI',.05,3,.45,2],
 ['hgt',.05,3,.55,2],
 ['Lipoprotein.A',.15,3,.65,2]])
inits.columns=['trait','left_asymp','inflection_pt','asymp_guess','slope']
inits=inits.set_index('trait')

In [2]:
#Define paramter bounds. These serve two purposes:
#1) parameters are kept within physical parameter ranges, e.g.,
#AUC<1, correlation<1, metrics grow with training size, etc.
#2) they stop the fitting routine if a parameter is running off to 
#infinity and the computation is taking a long time because of that.

#Fitting routine can be run without any bounds by simply replacing
#these arrays with empty arrays

#CACO
caco_bnds = [((0.3,0,-50,-10),(0.65,0.5,5,-0.5)),          
             ((0.3,-.5,-50,-10),(.7,0.5,5,-0.5)),
             ((0.3,-.5,-50,-10),(.7,0.5,5,-0.5)),
             ((0.3,-.5,-50,-10),(.7,0.5,5,-0.5)),
             ((0.3,0,0,-10),(0.65,0.4,5,-0.5))]
#CONT
cont_bnds = [((-0.1,0,-50,-10),(0.65,0.75,5,-0.5)),          
             ((-0.1,0,-50,-10),(.7,0.5,5,-0.5)),
             ((-0.1,0,-50,-10),(.7,0.5,5,-0.5)),         
             ((-0.1,0,-50,-10),(.7,0.5,5,-0.5)),
             ((-0.1,0,0,-10),(0.65,0.4,5,-0.5))]

In [ ]:
#Define fitting and analysis functions that are used below

def fitsingle(xs,ys,yerrs,func,init,**kwargs):
    opt_params={'bounds' : None, 'secret':13}
    for key, value in kwargs.items():
        if key in opt_params.keys():
            opt_params[key]=value
    if opt_params['bounds'] == None:
        return sp.optimize.curve_fit(func,xs,ys,p0=init,sigma=yerrs,absolute_sigma=True,maxfev = 4000)
    else:
        return sp.optimize.curve_fit(func,xs,ys,p0=init,sigma=yerrs,absolute_sigma=True,maxfev = 4000,bounds=opt_params['bounds'])

#func that gives chi2 function
def chisq(xs,ys,yerrs,result,numofparams,funcs):
    ind=0
    expect=[]
    for func in funcs: 
        expect.append(func(xs,*result[ind][0]))
        ind += 1
    chisq =np.array([*map(np.sum,((ys-expect)/yerrs)**2)])
    df = len(xs)-4
    chisqperdof = np.array([x / df for x in chisq])
    return chisq, df, chisqperdof

#func to build cholesky matrix
def cholesky(result,numofparams):
    numofmodels = len(result)
    #central values
    centers = np.empty([numofmodels,numofparams])
    for i in range(0,numofmodels):
        centers[i] = result[i][0]
    #param errors
    errs = np.empty([numofmodels,numofparams])
    for i in range(0,numofmodels):
        errs[i]=np.sqrt(np.diag(result[i][1]))
    #convert covariance to correlation matrix
    corr = np.empty([numofparams*numofmodels,numofparams])
    for i in range(0,numofmodels):
        temp=1/np.sqrt(np.diag(result[i][1]))
        corr[i*numofparams:(i*numofparams+numofparams)]=np.dot(np.dot(np.diagflat(temp),result[i][1]),np.diagflat(temp))
    #do cholesky decomp
    chol = np.empty([numofparams*numofmodels,numofparams])
    for i in range(0,numofmodels):
        chol[i*numofparams:(i*numofparams+numofparams)]=np.linalg.cholesky(corr[i*numofparams:(i*numofparams+numofparams)])
    return chol

#MCpts builds monte carlo data using the cholesky method while allowing parameters to vary as correlated gaussian normal distributions.
def MCpts(numofpts,numofmodels,numofparams,cholesky,result):
    centers = np.empty([numofmodels,numofparams])
    for i in range(0,numofmodels):
        centers[i] = result[i][0]
    errs = np.empty([numofmodels,numofparams])
    for i in range(0,numofmodels):
        errs[i]=np.sqrt(np.diag(result[i][1]))
    mcptstotal = numofpts
    mcpts=np.empty([numofmodels*mcptstotal,numofparams])
    for i in range(0,numofmodels):
        for j in range(0,mcptstotal):
            mcpts[i*mcptstotal+j] = centers[i]+np.multiply(np.dot( cholesky[i*numofparams:(i*numofparams+numofparams)],np.random.normal(loc=0.0, scale=1.0, size=(numofparams,1))).flatten(),errs[i])
    return mcpts

In [1]:
trait=['TRAIT NAME']

#Path to top working directory
topPATH = 'Path to working directory'
#Phenotype style: CACO (case control) or CONT
phenTYPE = 'CACO'
#path to different cohort group files, e.g., list of IDs of individuals who are siblings or a specific ancestry
cohortPATH='Path to cohort files'
#load sibling IDs. Sibling file is two columns of IDs where the columns represent sibling pairs. Each sibling pair appears only once. 
#NEED TO SET
sibs = pd.read_csv('Path to list of sibling IDs')
#covariates adjusted for: commonly PCA (PC vectors), year of birth (YOB), and sex
covTYPE = 'PCA_YOB_SEX'
#number of monte carlo samples  for fitting
n_mc=5000    
#Load list of train sizes #NEED TO SET
trainsizes=np.loadtxt('Path to list of training sizes used')
#load prediction metrics (e.g., AUC or correlation)
#Note there can be inherent uncertainty in how you choose the maximal value (i.e. hyperparameter) in the validation set.
#In the manuscript, the sib_met is selected by taking the hyperparameter for the maximal value in the 
#validation set and then using that hyperparameter to score the siblings or ancestry group.
metrics=np.empty((len(trainsizes),5))
for m in trainsizes:
    for i in range(1,6):
        sib_met = np.loadtxt('Path to metrics for siblings with files index by m value and CV-fold i')
        metrics[np.where(trainsizes==m),i-1] = sib_met          
pheno=pd.read_csv('Path to phenotype file with columns: individual ID, family ID, adjusted phenotype value')
#generate fitting data: train sizes vs mean (over CV-folds) metric value
xs=trainsizes
ys=np.mean(metrics,axis=1)
if gwasTYPE == 'CACO':
    #load the case control status
    caco = pd.read_csv('Path to full case control status values')
    #Subset down the full case control list to just siblings
    sib_caco = caco[caco[0].isin(sibs[0]) | caco[0].isin(sibs[1])]
    #total number of cases
    sibcas=sib_caco[2].sum()
    #total uncertainty on the average AUC value (computation explained in the manuscript). The first term comes from running
    #multiple CVs and the second term comes from the uncertainty of computing AUC with finite statistics 
    yerrs=np.sqrt(np.std(metrics,axis=1)**2+(1/(3*np.sqrt(sibcas)))**2)
else:
    #subset the full phenotype file down to just siblings
    sib_pheno=pheno[pheno[0].isin(sibs[0]) | pheno[0].isin(sibs[1])]
    #total uncertainty on the average correlation value (computation explained in the manuscript). The first term comes from running
    #multiple CVs and the second term comes from the uncertainty of computing correlation with finite statistics 
    yerrs=np.sqrt(np.std(metrics,axis=1)**2+(1-ys**2)/(len(sib_pheno)-2))
    
    
#initial guesses and make fits
#call values from tables
if trait in inits.index:
    left_asymp = inits.loc[trait]['left_asymp']; inflection_pt = inits.loc[trait]['inflection_pt']; 
    asymp_guess = inits.loc[trait]['asymp_guess']; slope = inits.loc[trait]['slope']
else: #if no values exist in table
    left_asymp = 0.5; inflection_pt = 3; asymp_guess = .65; slope = 2;
#convert initial guesses to parameter values for the various functions used
params4=[
     [left_asymp,asymp_guess-left_asymp,-slope,-inflection_pt],
     [asymp_guess/2,asymp_guess/2,slope,-inflection_pt],
     [asymp_guess/2,asymp_guess/2,slope,-inflection_pt],
     [asymp_guess/2,asymp_guess/2,slope,-inflection_pt]]
#load parameter bounds if using them
if gwasTYPE != 'CACO':
    bnds=cont_bnds
else:
    bnds=caco_bnds

####################################
####################################
### FITTING ROUTINE AND ANALYSIS ###
####################################
####################################

#run fitting routing
result4=[]
for i in range(0,len(funcs4)):
    if bnds[i]==None:
        result4.append(fitsingle(np.log10(xs),ys,yerrs,funcs4[i],params4[i])    )
    else:
        result4.append(fitsingle(np.log10(xs),ys,yerrs,funcs4[i],params4[i],bounds=bnds[i]))
#get chisq
chisq4 = chisq(np.log10(xs),ys,yerrs,result4,4,funcs4)
#build cholesky matrices
cho4 = cholesky(result4,4)
#build MC points
mcpts4 = MCpts(n_mc,len(funcs4),4,cho4,result4)
#filter out "unphysical" MC pts. in this case we are only using the condition b>0
altpts=[]
altlens=[]
for i in range(0,len(funcs4)):
    tmp=pd.DataFrame(mcpts4[i*n_mc:(i+1)*n_mc])
    altpts.append(tmp[(tmp[1]>0)&((tmp[0]+tmp[1])<1)].to_numpy())
    altlens.append(tmp[(tmp[1]>0)&((tmp[0]+tmp[1])<1)].shape[0])
altfirst = np.insert(np.cumsum(altlens),0,0)

#define asymptotic values as an average over MC asymptotes (THIS DEPENDS ON SPECIFIC FORM OF THE FUNCTIONS)  
asymp4 = np.empty([len(funcs4),2])
for i in range(0,len(funcs4)):
    temp = altpts[i][:,0]+altpts[i][:,1]
    asymp4[i,0]=np.mean(temp)
    asymp4[i,1]=np.std(temp)
        

#generate band data
nxs=500
tmpxs = np.linspace(0,8,int(nxs/2))
tmpxs = np.hstack((tmpxs,np.random.normal(loc=-result4[0][0][3],scale=1,size=int(nxs/2))))
tmpxs = np.sort(tmpxs)
#minmax4 will be the list of minimum and maximum values for the uncertainty bands generated with MC
#minmax4 will have 4 sets of minmax data, one for each function
minmax4=[]
df_asymp4 = np.empty([len(funcs4),2])
for func in funcs4:
    xs=np.log10(trainsizes)
    dof=len(xs)-4
    cs=[]
    for i in range(0,altlens[funcs4.index(func)]):
        cs.append(np.sum(((ys-func(xs,*altpts[funcs4.index(func)][i]))/yerrs)**2))
    df=pd.DataFrame(cs)

    # num of sigmas in each direction
    # 1 sigma = 1-0.682 = 0.318
    # 2 sigma = 1-0.954 = 0.046
    # 3 sigma = 1-0.996 = 0.004
    band_sigs = {'1sig': 0.318, '2sig':0.046,'3sig':0.004}
    cl = band_sigs[band_sig]
    #one sided cut
    cl_cut=df.quantile(q=1-cl).values[0]
    #indices with data within the confidence limit
    inds=df[df[0]<cl_cut].index.values

    tmp_minmax4=np.zeros((nxs,2))
    for j in range(0,nxs):
        tmp=[]
        for i in inds:
            tmp.append(func(tmpxs[j],*altpts[funcs4.index(func)][i]))
        #be careful of poor fit outliers
        tmp_minmax4[j,:] = [np.min(tmp),np.max(tmp)]
    minmax4.append(tmp_minmax4)
    
    #df_asymp4 is a dataframe with the (right side) asymptotic results for each function
    #averaged over all MC points in the confidence interval.
    tmp_asymp=altpts[funcs4.index(func)][inds].T
    df_asymp4[funcs4.index(func),0]=np.mean(tmp_asymp[0] + tmp_asymp[1])
    df_asymp4[funcs4.index(func),1]=np.std(tmp_asymp[0] + tmp_asymp[1])

NameError: name 'pd' is not defined